### How to use Jupyter Notebook:

Jupyter Notebook is split into cell blocks and can run a script one cell block at a time. 

To run the cells with code, click the cell then click "Run" on the toolbar above or hit Ctrl+Shift on the keyboard. Runable cells have the In[ ] on the left side.

See the general process of the code and common scripts in the "README.md" file and "CBECC-com python basics" Jupyter Notebook.

# Update VAV box inputs in Excel

This script extracts user inputs for all VAV boxes in the CBECC-com model into Excel, then allows the user to modify the inputs and re-import it back into Excel

*When modifying inputs in Excel, don't change the VAV box name or else the scripts can not properly identify the changes*

### Important Notes:
- Save this Jupyter Notebook file in the same folder as the CBECC-com file (.cibd16x or .xml). It is recommended to keep a backup file.
- Only hard-coded user inputs (red text inputs in CBECC=com) are extracted. If you want to extract CBECC-com autosized inputs, use "Pull Autosized VAV Capacities.ipynb" script.
- Does not work for active chilled beams or fan-powered boxes.
- Currently cannot update the Assigned AHU. Change the Assigned AHU, click and drag in CBECC-com.
- This script does not check for CBECC-com errors so make sure your excel sheet inputs are appropriate.

## Setting up the script
Enter the file name of the CBECC-com model you would like to modify and run the following cell block (Ctrl+Shift).

In [ ]:
# Enter the file name of the CBECC-com model. File paths or links do not work.
file = ('SRJC STEM Building Title 24.cibd16x')


# Importing necessary libraries and setting up xml root
from lxml import etree as ET
import pandas as pd
import numpy as np
from IPython.display import Markdown, display

filename = file.split('.')[0]
filetype = file.split('.')[1]

tree = ET.parse(file)
root = tree.getroot()

excel_file = filename + ' - VAVData.xlsx'

## Step 1: Extract VAV box inputs into Excel
Following script iterates through all Terminal Units and extracts the information into a dataframe then writes the dataframe into Excel.

The extracted excel file will be called "*filename* - VAVData.xlsx". Make sure to close the Excel file if re-running Step 1.

Run the following cell block. No changes to the code is necessary.

In [ ]:
### Parsing the model and creating dataframes

trml_unit_list = []

for unit in root.iter('TrmlUnit'):
    trml = {}
    
    trml['Terminal Unit Name'] = unit.find('Name').text
    trml['Type'] = unit.find('Type').text
    trml['Assigned AHU'] = unit.getparent().find('Name').text
    trml['Zone Served'] = unit.find('ZnServedRef').text
    trml['Primary Flow'] = float(unit.find('PriAirFlowMax').text)
    try:
        trml['Primary Min. Flow'] = float(unit.find('PriAirFlowMin').text)
    except:
        pass
    
    if trml['Type'] == 'VAVReheatBox':
        try:
            trml['Heating Air Flow'] = float(unit.find('HtgAirFlowMax').text)
        except:
            pass
        try:
            trml['Reheat Coil Name'] = unit.find('CoilHtg/Name').text
        except:
            pass
        try:
            trml['Reheat Coil Type'] = unit.find('CoilHtg/Type').text
        except:
            pass
        if trml['Reheat Coil Type'] == 'HotWater':
            try:
                trml['Hot Water Inlet Segment'] = unit.find('CoilHtg/FluidSegInRef').text
            except:
                pass
            try:
                trml['Hot Water Outlet Segment'] = unit.find('CoilHtg/FluidSegOutRef').text
            except:
                pass
            
        heating_cap = 0
        try:
            heating_cap += float(unit.find('CoilHtg/CapTotNetRtd').text)
        except:
            try:
                heating_cap += float(unit.find('CoilHtg/CapTotGrossRtd').text)
            except:
                pass
        if heating_cap == 0:
            pass
        else:
            trml['Reheat Coil Capacity'] = heating_cap
    trml_unit_list.append(trml)
    
trml_header = [
    'Terminal Unit Name', 'Type', 'Assigned AHU', 'Zone Served', 
    'Primary Flow', 'Primary Min. Flow', 'Heating Air Flow', 
    'Reheat Coil Name', 'Reheat Coil Type', 'Hot Water Inlet Segment', 
    'Hot Water Outlet Segment', 'Reheat Coil Capacity'
]
    
trml_df = pd.DataFrame(trml_unit_list, columns=trml_header)


# Writing dataframe into excel
writer = pd.ExcelWriter(excel_file, engine='xlsxwriter')
trml_df.to_excel(writer,sheet_name='Terminal Units')
writer.save()

## Step 2: Update the Excel Sheet
Open the "*filename* - VAVData.xlsx" in the same folder as this script. Make any changes in the Excel file *EXCEPT* for the VAV box name. 

If changing the VAV box type, make sure the appropriate inputs are filled in. For example, if changing from VAV with no reheat to VAV with reheat, make sure to also add the heating air flow and all the inputs required for the reheat coil.

As long as the VAV box names are not changed, Step 1 does not need to be re-run. Re-running Step 1 will reset any changes made in the Excel file. If you need to re-run Step 1 but want to keep the changes from excel, rename or move the original excel file.

## Step 3: Re-import Excel file updates back into CBECC-com:
Make sure to save and close the Excel file and run the following script. 

The new CBECC-com file will be called "*filename*_new".

Notes:
- The Excel sheet can be modified and re-imported again without re-running Step 1.
- Make sure to check for any warnings and errors immediately after running the code block.
- No changes to the code is necessary.

In [ ]:
### Re-parsing root file in case Excel file is attempted to be re-imported

tree = ET.parse(file)
root = tree.getroot()



### Creating a new dataframe from the updated excel sheet and modifying the xml file. 

trml_df_new=pd.read_excel(excel_file,sheet_name='Terminal Units', index_col=0)

for i,row in trml_df_new.iterrows():
    unit = root.xpath('.//TrmlUnit/Name[text()="{}"]'.format(row['Terminal Unit Name']))[0].getparent()
    
    unit.find('Type').text = str(row['Type'])
    unit.find('ZnServedRef').text = str(row['Zone Served'])
    unit.find('PriAirFlowMax').text = str(row['Primary Flow'])
    try:
        unit.find('PriAirFlowMin').text = str(row['Primary Min. Flow'])
    except:
        if row['Type'] != 'Uncontrolled':
            min_flow = ET.SubElement(unit, 'PriAirFlowMin')
            min_flow.text = str(row['Primary Min. Flow'])
    if row['Type'] == 'VAVReheatBox':
        try:
            unit.find('HtgAirFlowMax').text = str(row['Heating Air Flow'])
        except:
            heat_flow = ET.SubElement(unit, 'HtgAirFlowMax')
            if str(row['Heating Air Flow']) != 'nan':
                heat_flow.text = str(row['Heating Air Flow'])
            else:
                display(Markdown('### <span style="color:orange">Warning - Heating airflow rate for terminal unit {} is blank</span>'.format(row['Terminal Unit Name'])))
                pass
        
        if unit.find('CoilHtg') == None:
            HtgCoil = ET.SubElement(unit, 'CoilHtg')
        
        CoilHtg = unit.find('CoilHtg')
        
        try:
            CoilHtg.find('Name').text = str(row['Reheat Coil Name'])
        except:
            rhc_name = ET.SubElement(CoilHtg, 'Name')
            if str(row['Reheat Coil Name']) != 'nan':
                rhc_name.text = str(row['Reheat Coil Name'])
            else:
                display(Markdown('### <span style="color:crimson">ERROR! - Enter name for heating coil in terminal unit {}</span>'.format(row['Terminal Unit Name'])))
                break
        try:
            CoilHtg.find('Type').text = str(row['Reheat Coil Type'])
        except:
            rhc_type = ET.SubElement(CoilHtg, 'Type')
            if str(row['Reheat Coil Type']) != 'nan':
                rhc_type.text = str(row['Reheat Coil Type'])
            else:
                display(Markdown('### <span style="color:orange">Warning - Enter reheat coil type for {}</span>'.format(rhc_name.text)))
                pass
        
            if rhc_type.text == 'HotWater':
                try: 
                    CoilHtg.find('FluidSegInRef').text = str(row['Hot Water Inlet Segment'])
                except:
                    rhc_hwi = ET.SubElement(CoilHtg, 'FluidSegInRef')
                    if str(row['Hot Water Inlet Segment']) != 'nan':
                        rhc_hwi.text = str(row['Hot Water Inlet Segment'])
                    else:
                        display(Markdown('### <span style="color:orange">Warning - Hot water inlet fluid segment for {} is blank</span>'.format(rhc_name.text)))
                        pass
                
                try: 
                    CoilHtg.find('FluidSegOutRef').text = str(row['Hot Water Outlet Segment'])
                except:
                    rhc_hwi = ET.SubElement(CoilHtg, 'FluidSegOutRef')
                    if str(row['Hot Water Outlet Segment']) != 'nan':
                        rhc_hwi.text = str(row['Hot Water Outlet Segment'])
                    else:
                        display(Markdown('### <span style="color:orange">Warning - Hot water outlet fluid segment for {} is blank</span>'.format(rhc_name.text)))
                        pass
        try:
            CoilHtg.find('CapTotGrossRtd').text = str(row['Reheat Coil Capacity'])
        except:
            rhc_cap = ET.SubElement(CoilHtg, 'CapTotGrossRtd')
            if str(row['Reheat Coil Capacity']) != 'nan':
                rhc_cap.text = str(row['Reheat Coil Capacity'])
            else:
                display(Markdown('### <span style="color:orange">Warning - Reheat coil capacity for {} is blank</span>'.format(rhc_name.text)))
                pass
                
tree.write(filename + '_new.' + filetype, pretty_print=True)

## Check the newly created CBECC-com file

Open the new CBECC-com file and make sure the program can properly read the updated inputs. 

Re-run the cell blocks as necessary. If you to re-run this entire script based on the new CBECC-com file, move/delete the original and name the new file to the old file or change the file name input in Step 1.

### Future updates to add:

Allow the creation of new equipments from the updated Excel file.
Data validation in pandas to limit/lock excel cells.
Move the assigned AHU from Excel.